# Workflow con ZlightGBM

## Inicializacion

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
PARAM <- list()
PARAM$experimento <- "wfzr9321"
PARAM$semilla_primigenia <- 333133 # cambio semilla primigenia

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

# dataset <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

### Eliminacion de Features

In [ ]:
# Identificar columnas a eliminar
columnas_eliminar <- grep("prestamos_personales|limitecompra", names(dataset), value = TRUE)

# Visualizar las columnas que se van a eliminar
print(paste("Total de columnas a eliminar:", length(columnas_eliminar)))
print(columnas_eliminar)

In [ ]:
 # Eliminar las columnas
dataset[, (columnas_eliminar) := NULL]

dim(dataset)


### Data Quality

### NA si batch == 0

In [ ]:
#EXPLORO

res_wide <- dataset[
  , lapply(.SD, function(v) {
      # condición estricta: sin NAs y todos exactamente 0
      if (!all(!is.na(v))) return(FALSE)
      all(v == 0)
    }),
  by = foto_mes
]

cero_por_batch <- melt(
  res_wide,
  id.vars = "foto_mes",
  variable.name = "variable",
  value.name = "todo_cero"
)[todo_cero == TRUE, .(variable, foto_mes)]

cero_por_batch


In [ ]:
# APLICO
NA_of <- function(v){
  if (inherits(v, "integer64")) {
    if (requireNamespace("bit64", quietly = TRUE)) return(bit64::NA_integer64_)
    return(NA)  # fallback si no está bit64
  }
  if (is.integer(v))   return(NA_integer_)
  if (is.numeric(v))   return(NA_real_)
  if (is.logical(v))   return(NA)
  if (is.character(v)) return(NA_character_)
  # factor, Date, etc. → NA genérico
  NA
}

vars <- unique(cero_por_batch$variable)

for (var in vars) {
  meses <- cero_por_batch[variable == var, foto_mes]
  na_val <- NA_of(dataset[[var]])
  
  # comparación a 0 según tipo (para evitar coerciones raras)
  if (is.character(dataset[[var]])) {
    dataset[foto_mes %in% meses & get(var) == "0", (var) := na_val]
  } else {
    dataset[foto_mes %in% meses & get(var) == 0,  (var) := na_val]
  }
}


### monto NA si c == 0

In [ ]:
# Diccionario 1: (c -> m)  aplicar NA a m cuando c == 0/"0"

dict_c0_mNA <- list(
  c("ccaja_ahorro","mcaja_ahorro"),
  c("ccaja_ahorro","mcaja_ahorro_dolares"),
  c("ccajeros_propios_descuentos","mcajeros_propios_descuentos"),
  c("ccheques_depositados_rechazados","mcheques_depositados_rechazados"),
  c("ccheques_emitidos_rechazados","mcheques_emitidos_rechazados"),
  c("ccomisiones_mantenimiento","mcomisiones_mantenimiento"),
  c("ccuenta_corriente","mcuenta_corriente"),
  c("cextraccion_autoservicio","mextraccion_autoservicio"),
  c("cforex_buy","mforex_buy"),
  c("cforex_sell","mforex_sell"),
  c("cinversion1","minversion1_dolares"),
  c("cinversion1","minversion1_pesos"),
  c("cinversion2","minversion2"),
  c("cpagodeservicios","mpagodeservicios"),
  c("cpagomiscuentas","mpagomiscuentas"),
  c("cplazo_fijo","mplazo_fijo_dolares"),
  c("cplazo_fijo","mplazo_fijo_pesos"),
  c("cprestamos_hipotecarios","mprestamos_hipotecarios"),
  c("cprestamos_prendarios","mprestamos_prendarios"),
  c("ctarjeta_visa", "Visa_mlimitecompra"),
  c("ctarjeta_master", "Master_mlimitecompra")
)

# Diccionario 2: (c -> m) aplicar NA a m cuando (c == 0/"0"  y  m == 0/"0")
dict_c0_m0_mNA <- list(
  c("ccheques_depositados","mcheques_depositados"),
  c("ccheques_emitidos","mcheques_emitidos")
)

# Diccionario 3: ([c1,c2] -> [m1,m2]) aplicar NA a ambos m cuando (c1 == 0/"0"  y  c2 == 0/"0")
dict_c0c0_mNAmNA <- list(
  list(c_cols = c("cpayroll_trx","cpayroll2_trx"),
       m_cols = c("mpayroll","mpayroll2"))
)

# Helpers
NA_of <- function(v){
  if (inherits(v, "integer64")) return(if (requireNamespace("bit64", quietly=TRUE)) bit64::NA_integer64_ else NA)
  if (is.integer(v))   return(NA_integer_)
  if (is.numeric(v))   return(NA_real_)
  if (is.logical(v))   return(NA)
  if (is.character(v)) return(NA_character_)
  NA
}

is_zero <- function(x){
  if (is.character(x) || is.factor(x)) x == "0" else x == 0
}

### Feature Engineering Intra-Mes

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

### lags deltas

no laguear las continuas cuando haya rankings

In [ ]:
# Creacion de LAGs (excluí monetarias originales)
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy(setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria"
   # , variables_continuas
    )
))

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Sys.time()

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [ ]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

### Feature Engineering RF

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}

In [ ]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [ ]:
# Feature Engineering agregando variables de Random Forest
#  aqui es donde se hace el trabajo
AgregaVarRandomForest()

Sys.time()
sum(startsWith(names(dataset), "rf_"))

In [ ]:
ncol(dataset)
colnames(dataset)

# Produccion

In [ ]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)  
PARAM$train_final$false_future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912
, 202001, 202002, 202003
, 202004, 202005, 202006, 202007, 202008, 202009, 202010
, 202011, 202012
, 202101, 202102, 202103, 202104
, 202105, 202106
)

PARAM$train_final$undersampling <- 0.1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

## Canaritos

In [ ]:
cat("Columnas ANTES:", ncol(dataset_train_final), "\n")

PARAM$qcanaritos <- 5L
cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)
for(i in seq(PARAM$qcanaritos)) {
  dataset_train_final[, paste0("canarito_", i) := runif(filas)]
}
cols_canaritos <- setdiff(colnames(dataset_train_final), cols0)
setcolorder(dataset_train_final, c(cols_canaritos, cols0))


cat("Canaritos creados:", length(cols_canaritos), "\n")
cat("Columnas DESPUÉS:", ncol(dataset_train_final), "\n")


### Final Models

In [ ]:
# Semillerio Final APO
PARAM$train_final$APO <- 5
PARAM$train_final$ksemillerio  <- 20

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)

In [ ]:
# AGREGADO PARA APO porque si no esto estaría definido en la bayesiana
library(primes)
primos <- generate_primes(min = 100000, max = 1000000)
# =================== hasta acá

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )] # APO
PARAM$train_final$semillas

In [ ]:
# dejo los datos en formato LightGBM

campos_buenos <- setdiff(colnames(dataset_train_final), c("clase_ternaria","clase01","training"))


dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
PARAM$lgbm <- list(
  boosting          = "gbdt",
  objective         = "binary",
  metric            = "custom",
  first_metric_only = FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise    = TRUE,
  verbosity         = -100,
  seed              = PARAM$semilla_primigenia,
  max_bin           = 31L,
  min_data_in_leaf  = 200L,  #default era 20
  num_iterations    = 64L,  #default era 9999L
  num_leaves        = 9999L,
  learning_rate     = 1.0,
  feature_fraction  = 0.50,
  canaritos         = PARAM$qcanaritos,
  gradient_bound    = 0.4    
)


In [ ]:
# Para compatibilidad con el bucle de semillerio de workflowtest:
PARAM$train_final$param_mejores <- PARAM$lgbm

## modelitos

In [ ]:
# FINAL TRAIN

dir.create("modelitos", showWarnings = FALSE)

param_completo <- copy(PARAM$train_final$param_mejores)

for (sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)

  if (!file.exists(arch_modelo)) {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data  = dtrain_final,
      param = param_completo
    )

    lgb.save(modelito, filename = arch_modelo)
    rm(modelito); gc()
    cat(" Modelo guardado seed", sem, "\n")
  } else {
    cat("Ya existía, salto seed", sem, "\n")
  }
}
cat("\nListo. Si no había pendientes, no se recalculó nada.\n")


### Scoring producción

Se hace el predict() del modelo en los datos del futuro

In [ ]:
# SCORING REAL

PARAM$train_final$future <- c(202108)

dfuture <- dataset[foto_mes %in% PARAM$train_final$future]

cat("Columnas ANTES:", ncol(dfuture), "\n")

######################

cols_fut_0 <- copy(colnames(dfuture))

filas_fut <- nrow(dfuture)
for(i in seq(PARAM$qcanaritos)) {
  dfuture[, paste0("canarito_", i) := runif(filas_fut)] 
}

cols_canaritos_fut <- setdiff(colnames(dfuture), cols_fut_0)
setcolorder(dfuture, c(cols_canaritos_fut, cols_fut_0))

cat("Canaritos creados:", length(cols_canaritos_fut), "\n")
cat("Columnas DESPUÉS:", ncol(dfuture), "\n")

#########################

mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0L

for(sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)
  modelo_final <- lgb.load(arch_modelo)
  vpred_individual <- predict(modelo_final, mfuture)

  tb_pred_seed <- dfuture[, .(numero_de_cliente, foto_mes)]
  tb_pred_seed[, prob := vpred_individual ]
  fwrite(tb_pred_seed,
         file = sprintf("prediccion_future_real_%s_%s.txt", PARAM$experimento, sem),
         sep  = "\t")

  vpred_acum <- vpred_acum + vpred_individual
  qacumulados <- qacumulados + 1L
}

vpred_acum <- vpred_acum / qacumulados

tb_prediccion <- dfuture[, .(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]
fwrite(tb_prediccion,
       file = sprintf("prediccion_future_real_%s.txt", PARAM$experimento),
       sep  = "\t")

# YML

In [ ]:
cat("\n=== Guardando parámetros del experimento ===\n")
if (!requireNamespace("yaml", quietly = TRUE)) install.packages("yaml")

# Construir primero la versión "plana"
PARAM_export <- list(
  experimento  = PARAM$experimento,
  train_final  = list(
    future        = PARAM$train_final$future,
    training      = PARAM$train_final$training,
    undersampling = PARAM$train_final$undersampling,
    semillas      = PARAM$train_final$semillas,
    param_mejores = PARAM$train_final$param_mejores
  ),
  cortes         = if ("cortes" %in% names(PARAM)) PARAM$cortes else NULL,
  qcanaritos     = if ("qcanaritos" %in% names(PARAM)) PARAM$qcanaritos else NULL,
  campos_buenos  = if (exists("campos_buenos")) campos_buenos else NULL
)

nombre_archivo_param <- sprintf("PARAM_plano_exp_%s.yml", PARAM$experimento)
yaml::write_yaml(PARAM_export, file = nombre_archivo_param)
cat("Archivo de parámetros guardado exitosamente en:", nombre_archivo_param, "\n")
